# Testing Different Preprocessing Techniques (Sandbox Env)

## Data preprocessing
Image sharpening

In [ ]:
import cv2
import os
import numpy as np

# Directories
input_dir = '/content/Dataset/Dataset/Images'
output_dir = '/content/Dataset/Dataset/Enhanced_Images'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Sharpening kernel
kernel = np.array([[-1, -1, -1],
                   [-1,  9, -1],
                   [-1, -1, -1]])

# Process each image
for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Read the image
        img_path = os.path.join(input_dir, filename)
        image = cv2.imread(img_path)

        # Apply the sharpening kernel
        sharpened = cv2.filter2D(image, -1, kernel)

        # Save the enhanced image
        cv2.imwrite(os.path.join(output_dir, filename), sharpened)

print("Image enhancement complete.")


Image enhancement complete.


### Splitting enhanced images
comment these cells out if you don't want to use this image sharpening pipeline

In [ ]:
#clean up previous files
#!rm -r /content/Dataset/Dataset/labels
#!rm -r /content/Dataset/Dataset/images

!rm -r /content/Dataset/Dataset/Enhanced_Images/
!rm -r /content/Dataset/Dataset/Enhanced_labels

rm: cannot remove '/content/Dataset/Dataset/labels': No such file or directory


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

#AMMAN
source_images_dir = '/content/Dataset/Dataset/Enhanced_Images'
source_labels_dir = '//content/Dataset/Dataset/Labels'
train_images_dir = '/content/Dataset/Dataset/Enhanced_Images/train'
val_images_dir = '/content/Dataset/Dataset/Enhanced_Images/val'
train_labels_dir = '/content/Dataset/Dataset/Enhanced_labels/train'
val_labels_dir = '/content/Dataset/Dataset/Enhanced_labels/val'

# Create directories for train and validation sets
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Get all image filenames (assuming they are TIFF files)      MAXAR
#all_images = [f for f in os.listdir(source_images_dir) if f.endswith('.tif')]

# Get all image filenames (assuming they are png files)     Amman
all_images = [f for f in os.listdir(source_images_dir) if f.endswith('.png')]

# Split the dataset
train_images, val_images = train_test_split(all_images, test_size=0.2, random_state=42)

# Copy image files to their respective directories
for image in train_images:
    shutil.copy(os.path.join(source_images_dir, image), train_images_dir)
    # Copy corresponding label file
    label_file = image.replace('.png', '.txt')
    shutil.copy(os.path.join(source_labels_dir, label_file), train_labels_dir)

for image in val_images:
    shutil.copy(os.path.join(source_images_dir, image), val_images_dir)
    # Copy corresponding label file
    label_file = image.replace('.png', '.txt')
    shutil.copy(os.path.join(source_labels_dir, label_file), val_labels_dir)


In [ ]:
!cp -rf /content/Dataset/Dataset/Enhanced_labels/train/* /content/Dataset/Dataset/Enhanced_Images/train/
!cp -rf /content/Dataset/Dataset/Enhanced_labels/val/* /content/Dataset/Dataset/Enhanced_Images/val/


In [ ]:
import yaml

# Define the data
data = {
    'train': '/content/Dataset/Dataset/Enhanced_Images/train',
    'val': '/content/Dataset/Dataset/Enhanced_Images/val',
    'nc': 1,
    'names': ['0']  # 'names' should be a list of class names
}

# Write the data to a YAML file
with open('/content/yolov5/Sharpened_dataset.yaml', 'w') as file:
    yaml.dump(data, file)


In [ ]:
cat /content/yolov5/Sharpened_dataset.yaml

label_dir: /content/Dataset/Dataset/Enhanced_labels
names:
- '0'
nc: 1
train: /content/Dataset/Dataset/Enhanced_Images/train
val: /content/Dataset/Dataset/Enhanced_Images/val


## Labeled images - no preprocessing

In [ ]:
#!python train.py --img 256 --batch 25 --epochs 120 --data dataset.yaml --weights yolov5s --cfg models/yolov5s.yaml
#!python train.py --img 256 --batch 60 --epochs 199 --data dataset.yaml --weights yolov5s --cfg models/yolov5s.yaml
#!python train.py --img 256 --batch 80 --epochs 220 --data dataset.yaml --weights yolov5s --cfg models/yolov5s.yaml
#!python train.py --img 256 --batch 120 --epochs 300 --data dataset.yaml --weights yolov5s --cfg models/yolov5s.yaml #runs\traim\exp
!python train.py --img 256 --batch 240 --epochs 350 --data dataset.yaml --weights yolov5s --cfg models/yolov5s.yaml #runs\traim\exp2
#!python train.py --img 256 --batch 160 --epochs 320 --data dataset.yaml --weights yolov5s --cfg models/yolov5s.yaml

2024-01-17 13:12:27.868899: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-17 13:12:27.868953: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-17 13:12:27.870671: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s, cfg=models/yolov5s.yaml, data=dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=350, batch_size=240, imgsz=256, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, si

### Evaluate Model

In [ ]:
# Remember to change the model in weights and the dataset in data to reflect the actual testing elements
!python val.py \
    --weights /content/yolov5/runs/train/exp2/weights/best.pt \
    --data /content/yolov5/dataset.yaml \
    --img 256 \
    --iou-thres 0.5 --conf-thres 0.5 \
    --task val \
    --verbose \
    --augment \
    --save-txt \
    --save-conf \
    --save-json

    #                   all        181        519      0.875      0.826      0.892      0.598


val: data=/content/yolov5/dataset.yaml, weights=['/content/yolov5/runs/train/exp2/weights/best.pt'], batch_size=32, imgsz=256, conf_thres=0.5, iou_thres=0.5, max_det=300, task=val, device=, workers=8, single_cls=False, augment=True, verbose=True, save_txt=True, save_hybrid=False, save_conf=True, save_json=True, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
WARNING ⚠️ confidence threshold 0.5 > 0.001 produces invalid results
YOLOv5 🚀 v7.0-277-g703d37e Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs

Dataset not found ⚠️, missing paths ['/content/Dataset/Dataset/images/val']
Traceback (most recent call last):
  File "/content/yolov5/val.py", line 430, in <module>
    main(opt)
  File "/content/yolov5/val.py", line 401, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return 

## Added sharpening to images

In [ ]:
!python train.py --img 256 --batch 120 --epochs 300 --data Sharpened_dataset.yaml --weights yolov5s --cfg models/yolov5s.yaml ##runs\traim\exp7

2024-01-17 15:15:34.599416: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-17 15:15:34.599475: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-17 15:15:34.601621: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s, cfg=models/yolov5s.yaml, data=Sharpened_dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=120, imgsz=256, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale

### Evaluate Model

In [ ]:
!python val.py \
    --weights /content/yolov5/runs/train/exp7/weights/best.pt \
    --data /content/yolov5/Sharpened_dataset.yaml \
    --img 256 \
    --iou-thres 0.5 --conf-thres 0.5 \
    --task val \
    --verbose \
    --augment \
    --save-txt \
    --save-conf \
    --save-json

val: data=/content/yolov5/Sharpened_dataset.yaml, weights=['/content/yolov5/runs/train/exp7/weights/best.pt'], batch_size=32, imgsz=256, conf_thres=0.5, iou_thres=0.5, max_det=300, task=val, device=, workers=8, single_cls=False, augment=True, verbose=True, save_txt=True, save_hybrid=False, save_conf=True, save_json=True, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
WARNING ⚠️ confidence threshold 0.5 > 0.001 produces invalid results
YOLOv5 🚀 v7.0-277-g703d37e Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/Dataset/Dataset/Enhanced_Images/val.cache... 181 images, 1 backgrounds, 0 corrupt: 100% 181/181 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 6/6 [00:04<00:00,  1.22it/s]
                   all        181        519      0.864      0.829      0.882      0.584
Speed: 0.0ms

## Added Color Space Adjustment and Contrast Enhancement


In [ ]:
!rm -r /content/Dataset/Dataset/Contrast_Enhanced_Images

In [ ]:
import cv2
import os

# Define input and output directories
input_dir = '/content/Dataset/Dataset/Images'
output_dir = '/content/Dataset/Dataset/Contrast_Brightness_Enhanced_Images'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def enhance_contrast_brightness(image, alpha=1.5, beta=30):
    # Increase contrast and brightness
    enhanced_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

    return enhanced_image

# Process each image in the input directory
for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Read the image
        img_path = os.path.join(input_dir, filename)
        image = cv2.imread(img_path)

        # Enhance contrast and brightness
        contrast_brightness_enhanced_image = enhance_contrast_brightness(image)

        # Save the enhanced image
        cv2.imwrite(os.path.join(output_dir, filename), contrast_brightness_enhanced_image)

print("Contrast and brightness enhancement complete.")


Contrast and brightness enhancement complete.


Splitting the data

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define your dataset directories
source_images_dir = '/content/Dataset/Dataset/Contrast_Brightness_Enhanced_Images'
source_labels_dir = '/content/Dataset/Dataset/Labels'
train_images_dir = '/content/Dataset/Dataset/Contrast_Brightness_Enhanced_Images/train'
val_images_dir = '/content/Dataset/Dataset/Contrast_Brightness_Enhanced_Images/val'

# Create directories for train and validation sets
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)

# Get all image filenames (assuming they are PNG files)
all_images = [f for f in os.listdir(source_images_dir) if f.endswith('.png')]

# Split the dataset
train_images, val_images = train_test_split(all_images, test_size=0.2, random_state=42)

# Copy image files and corresponding label files to their respective directories
for image in train_images:
    shutil.copy(os.path.join(source_images_dir, image), train_images_dir)
    # Copy corresponding label file
    label_file = image.replace('.png', '.txt')
    shutil.copy(os.path.join(source_labels_dir, label_file), train_images_dir)

for image in val_images:
    shutil.copy(os.path.join(source_images_dir, image), val_images_dir)
    # Copy corresponding label file
    label_file = image.replace('.png', '.txt')
    shutil.copy(os.path.join(source_labels_dir, label_file), val_images_dir)


### Testing splitting based on number of objects

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
import random

# Define your dataset directories
source_images_dir = '/content/Dataset/Dataset/Contrast_Brightness_Enhanced_Images'
source_labels_dir = '/content/Dataset/Dataset/Labels'
train_images_dir = '/content/Dataset/Dataset/Contrast_Brightness_Enhanced_Images/train'
val_images_dir = '/content/Dataset/Dataset/Contrast_Brightness_Enhanced_Images/val'

# Create directories for train and validation sets
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)

# Get all image filenames (assuming they are PNG files)
all_images = [f for f in os.listdir(source_images_dir) if f.endswith('.png')]

# Calculate the total number of labeled objects in each image
image_objects_count = {}
for image in all_images:
    label_file = os.path.join(source_labels_dir, image.replace('.png', '.txt'))
    with open(label_file, 'r') as f:
        lines = f.readlines()
        image_objects_count[image] = len(lines)

# Sort the images based on the total number of objects (ascending order)
sorted_images = sorted(all_images, key=lambda x: image_objects_count[x])

# Calculate the number of images for validation (20% of the total images)
num_val_images = int(0.2 * len(sorted_images))

# Randomly select images for validation
val_images = random.sample(sorted_images, num_val_images)

# Copy image files and corresponding label files to their respective directories
for image in sorted_images:
    if image in val_images:
        shutil.copy(os.path.join(source_images_dir, image), val_images_dir)
        # Copy corresponding label file
        label_file = image.replace('.png', '.txt')
        shutil.copy(os.path.join(source_labels_dir, label_file), val_images_dir)
    else:
        shutil.copy(os.path.join(source_images_dir, image), train_images_dir)
        # Copy corresponding label file
        label_file = image.replace('.png', '.txt')
        shutil.copy(os.path.join(source_labels_dir, label_file), train_images_dir)


In [ ]:
#download to validate process
import shutil

# Define the directory to be zipped
directory_to_zip = '/content/Dataset/Dataset/Contrast_Brightness_Enhanced_Images'

# Define the output zip file name
output_zip_filename = '/content/Contrast_Brightness_Enhanced_Images.zip'

# Create a zip file
shutil.make_archive(output_zip_filename, 'zip', directory_to_zip)

print(f"Zipped directory '{directory_to_zip}' to '{output_zip_filename}'")


Zipped directory '/content/Dataset/Dataset/Contrast_Brightness_Enhanced_Images' to '/content/Contrast_Brightness_Enhanced_Images.zip'


In [ ]:
import yaml

# Define the data
data = {
    'train': '/content/Dataset/Dataset/Contrast_Brightness_Enhanced_Images/train',
    'val': '/content/Dataset/Dataset/Contrast_Brightness_Enhanced_Images/val',
    'nc': 1,
    'names': ['0']  # 'names' should be a list of class names
}

# Write the data to a YAML file
with open('/content/yolov5/Cntrst_Brt_dataset.yaml', 'w') as file:
    yaml.dump(data, file)


In [ ]:
cat /content/yolov5/Cntrst_Brt_dataset.yaml

names:
- '0'
nc: 1
train: /content/Dataset/Dataset/Contrast_Brightness_Enhanced_Images/train
val: /content/Dataset/Dataset/Contrast_Brightness_Enhanced_Images/val


In [ ]:
!python train.py --img 256 --batch 120 --epochs 300 --data Cntrst_Brt_dataset.yaml --weights yolov5s --cfg models/yolov5s.yaml ##runs\traim\exp8

2024-01-17 17:14:42.925783: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-17 17:14:42.925850: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-17 17:14:42.927789: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s, cfg=models/yolov5s.yaml, data=Cntrst_Brt_dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=120, imgsz=256, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scal